In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [20]:
data = pd.read_stata(r"D:\Liling\OneDrive - University of Pittsburgh\master_thesis\data\original.dta")

In [21]:
data1 = pd.read_csv(r"D:\Liling\OneDrive - University of Pittsburgh\master_thesis\q_learning_discrete_state\data\data_all_15.csv")

In [22]:
data_change_noscale = pd.read_csv(r"D:\Liling\OneDrive - University of Pittsburgh\master_thesis\rl_v1\data\data_all.csv")

In [23]:
#hosp_id_all = (ori_data_all.hosp_id).unique().tolist()
train_ids = pd.read_csv(r'D:\Liling\OneDrive - University of Pittsburgh\master_thesis\q_learning_discrete_state\data\train_id.csv')
test_ids =pd.read_csv(r'D:\Liling\OneDrive - University of Pittsburgh\master_thesis\q_learning_discrete_state\data\test_id.csv')


In [5]:
data.sofa_total

0          5.0
1          4.0
2          3.0
3          3.0
4          3.0
          ... 
309835     4.0
309836    12.0
309837     8.0
309838     8.0
309839     8.0
Name: sofa_total, Length: 309840, dtype: float64

In [33]:
state_750_new = pd.read_csv(r'D:\Liling\OneDrive - University of Pittsburgh\master_thesis\rl_v1\data_state\States.csv')

data1["new_state"] = state_750_new['State_0750'].tolist()


data1["new_next_state"] = 0

for i in data1.index:
  if i==0:
    continue
  else:
    data1.loc[i-1,"new_next_state"]=data1.loc[i,"new_state"]

In [6]:
max( data_change_noscale['vaso_change'])

7.599999904632568

In [7]:
data1['vaso_change'] = data_change_noscale['vaso_change'].tolist()

In [8]:
data1['norepi_equiv'] = data_change_noscale['norepi_equiv'].tolist()
data2 =data1[['hosp_id', 'interval','sofa_total','norepi_equiv','lactate','action', 'dead_90', 'reward_orig',
       'reward_intm', 'state', 'next_state',"new_state","new_next_state",'last_record', 'penalty_reward1',
       'penalty_reward2', 'vaso_change']]
data2 = data2.rename(columns={'penalty_reward1': 'reward_p15', 'penalty_reward2': 'reward_p5'})

KeyError: "['new_state', 'new_next_state'] not in index"

In [40]:
train_df = data2[data2.hosp_id.isin(train_ids.hosp_id)]

In [41]:
train_df.shape

(247629, 17)

In [42]:
max(data_change_noscale.vaso_change)

7.599999904632568

In [43]:
clin_p_old = np.zeros((750,25),dtype=float)
clin_p_new = np.zeros((750,25),dtype=float)

for row_index in train_df.index:
  state_old = int(train_df.loc[row_index,"state"]-1)
  state_new = int(train_df.loc[row_index,"new_state"]-1)
  action = int(train_df.loc[row_index,"action"]-1)
  clin_p_old[state_old][action] += 1
  clin_p_new[state_new][action] += 1
for i in [i for i in range(750)]:
  if sum(clin_p_old[i]) !=0:
    clin_p_old[i] /= sum(clin_p_old[i])
  if sum(clin_p_new[i]) !=0:
    clin_p_new[i] /= sum(clin_p_new[i])

In [44]:
clin_p_old

array([[0.21848739, 0.01680672, 0.00420168, ..., 0.0210084 , 0.03781513,
        0.03361345],
       [0.53965785, 0.00466563, 0.00311042, ..., 0.00466563, 0.        ,
        0.        ],
       [0.41666667, 0.00245098, 0.00245098, ..., 0.00245098, 0.00735294,
        0.00245098],
       ...,
       [0.44466019, 0.00194175, 0.00194175, ..., 0.        , 0.        ,
        0.        ],
       [0.67763158, 0.00328947, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.5655527 , 0.01285347, 0.00771208, ..., 0.00514139, 0.        ,
        0.        ]])

In [45]:
clin_p_new

array([[0.34644195, 0.00374532, 0.        , ..., 0.00749064, 0.00187266,
        0.        ],
       [0.37087912, 0.01648352, 0.01923077, ..., 0.00824176, 0.01098901,
        0.        ],
       [0.54356846, 0.02074689, 0.00207469, ..., 0.00207469, 0.00414938,
        0.        ],
       ...,
       [0.59677419, 0.00645161, 0.00645161, ..., 0.        , 0.00322581,
        0.        ],
       [0.3470437 , 0.05398458, 0.02056555, ..., 0.01028278, 0.01799486,
        0.00257069],
       [0.49425287, 0.00862069, 0.00287356, ..., 0.00574713, 0.00287356,
        0.        ]])

In [73]:
pd.DataFrame(clin_p_old).to_csv("data/clin_p_old.csv", index=False)
pd.DataFrame(clin_p_new).to_csv("data/clin_p_new.csv", index=False)


In [81]:
#data2["reward_p5_old"] = 0
#data2["reward_p15_old"] = 0
#data2["reward_p10_old"] = 0
#data2["reward_p5_new"] = 0
#data2["reward_p15_new"] = 0
#data2["reward_p10_new"] = 0
#for row_id in data2.index:
#  state_new = int(data2.loc[row_id,"new_state"]-1)
#  state_old = int(data2.loc[row_id,"state"]-1)
#  action = int(data2.loc[row_id,"action"]-1)
#  if clin_p_old[state_old][action]==0.0:
#    data2.loc[row_id,"reward_p5_old"] = -5
#    data2.loc[row_id,"reward_p10_old"] = -10
#    data2.loc[row_id,"reward_p15_old"] = -15
#  if clin_p_new[state_new][action]==0.0:
#    data2.loc[row_id,"reward_p5_new"] = -5
#    data2.loc[row_id,"reward_p10_new"] = -10
#    data2.loc[row_id,"reward_p15_new"] = -15
#  if data2.loc[i,"reward_orig"] !=0:
#    data2.loc[i,"reward_p5_old"] = data2.loc[i,"reward_orig"]
#    data2.loc[i,"reward_p10_old"] = data2.loc[i,"reward_orig"]
#    data2.loc[i,"reward_p15_old"] = data2.loc[i,"reward_orig"]  
#    data2.loc[i,"reward_p5_new"] = data2.loc[i,"reward_orig"]
#    data2.loc[i,"reward_p10_new"] = data2.loc[i,"reward_orig"]
#    data2.loc[i,"reward_p15_new"] = data2.loc[i,"reward_orig"]

C:\Users\lulil\Desktop\fastbook\myenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\lulil\Desktop\fastbook\myenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\lulil\Desktop\fastbook\myenv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [43]:
action_map = {}
count = 1
for iv in range(5):
    for vaso in range(5):
        action_map[(iv,vaso)] = count
        count += 1

action_map_1 = { action_map[k]:k for k in action_map}

action_map_2 = {key:value[0] for key, value in action_map_1.items()}
action_map_3 = {key:value[1] for key, value in action_map_1.items()}
data2['iv']=data2['action'].map(action_map_2)

data2['vaso']=data2['action'].map(action_map_3)

C:\Users\lulil\Desktop\fastbook\myenv\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\lulil\Desktop\fastbook\myenv\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [49]:
data2.tail()

,hosp_id,interval,sofa_total,norepi_equiv,lactate,action,dead_90,reward_orig,reward_intm,state,next_state,last_record,reward_p15,reward_p5,vaso_change,iv,vaso,reward_vaso5,reward_vaso10,reward_vaso15
309835,zzyOyK/jdML4Z0dRBUnWiFSr+fXITJMpn46RpEQblE8=,14.0,0.181818,0.00,0.249855,21.0,1.0,0,-0.303030,248,67,0,-15,-5,0.00,4,0,0,0,0
309836,zzyOyK/jdML4Z0dRBUnWiFSr+fXITJMpn46RpEQblE8=,15.0,0.545455,0.15,0.249855,3.0,1.0,0,0.151515,67,67,0,0,0,0.15,0,2,0,0,0
309837,zzyOyK/jdML4Z0dRBUnWiFSr+fXITJMpn46RpEQblE8=,16.0,0.363636,0.20,0.249855,4.0,1.0,0,-0.166667,67,32,0,0,0,0.05,0,3,0,0,0
309838,zzyOyK/jdML4Z0dRBUnWiFSr+fXITJMpn46RpEQblE8=,17.0,0.363636,0.20,0.249855,4.0,1.0,-15,-15.000000,32,67,0,-15,-15,0.00,0,3,-15,-15,-15
309839,zzyOyK/jdML4Z0dRBUnWiFSr+fXITJMpn46RpEQblE8=,18.0,0.363636,0.20,0.249855,4.0,1.0,0,0.000000,67,0,1,0,0,0.00,0,3,0,0,0


In [46]:
a = data2['norepi_equiv'][data2['vaso'] == 1]
action1=[round(a.min(),4),round(a.max(),4),round(a.median(),4)]
b = data2['norepi_equiv'][data2['vaso'] == 2]
action2=[round(b.min(),4),round(b.max(),4),round(b.median(),4)]
c = data2['norepi_equiv'][data2['vaso'] == 3]
action3=[round(c.min(),4),round(c.max(),4),round(c.median(),4)]
d = data2['norepi_equiv'][data2['vaso'] == 4]
action4=[round(d.min(),4),round(d.max(),4),round(d.median(),4)]
print(action1, action2,action3, action4)

[0.001, 0.09, 0.04] [0.09, 0.2, 0.15] [0.2, 0.5, 0.32] [0.501, 11.0, 1.08]


In [48]:
data2['reward_vaso5'] =0
data2.loc[data2.vaso_change>1.08,"reward_vaso5"] = -5
data2['reward_vaso10'] =0
data2.loc[data2.vaso_change>1.08,"reward_vaso10"] = -10
data2['reward_vaso15'] =0
data2.loc[data2.vaso_change>1.08,"reward_vaso15"] = -15
for i in data2.index:
  if data2.loc[i,"reward_orig"] !=0:
    data2.loc[i,"reward_vaso5"] = data2.loc[i,"reward_orig"]
    data2.loc[i,"reward_vaso10"] = data2.loc[i,"reward_orig"]
    data2.loc[i,"reward_vaso15"] = data2.loc[i,"reward_orig"]

In [47]:
n = 0
for row_id in train_df.index.tolist():
  state = int(train_df.loc[row_id,"new_state"])
  action = int(train_df.loc[row_id,"action"])
  if clin_p_new[state-1][action-1]==0.0:
    n +=1
print(n)     

0


In [54]:
data2['sofa_total']=data.sofa_total.tolist()
data2['lactate']=data.lactate.tolist()

In [55]:
# add rewards
c01 = -0.1/4
c11 = -0.5/4
c21 = -2

c02 = -0.1/2
c12 = -0.5/2
c22 = -2
data2['reward_intm1'] = 0
data2['reward_intm2'] = 0
for i in data2.index:
    if i == 0:
        continue
    if data2.loc[i, 'hosp_id'] == data2.loc[i-1, 'hosp_id']:
        sofa_cur = data2.loc[i,'sofa_total']
        sofa_prev = data2.loc[i-1,'sofa_total']
        lact_cur = data2.loc[i,'lactate']
        lact_prev = data2.loc[i-1,'lactate']
        reward1 = 0
        reward2 = 0
        if sofa_cur == sofa_prev and sofa_cur != 0:
            reward1 += c01
            reward2 += c02
        reward1 += c11*(sofa_cur-sofa_prev)
        reward1 += c21*np.tanh(lact_cur - lact_prev)
        reward2 += c12*(sofa_cur-sofa_prev)
        reward2 += c22*np.tanh(lact_cur - lact_prev)
        data2.loc[i-1,'reward_intm1'] = reward1
        data2.loc[i-1,'reward_intm2'] = reward2


        
    if i % 10000 == 0:
        print(i)

# if orig_df.ix[len(orig_df)-1, 'died_in_hosp'] == 1:
#     orig_df.ix[len(orig_df)-1, 'shaped_reward'] = -100
# elif orig_df.ix[len(orig_df)-1, 'died_in_hosp'] == 0:
#      orig_df.ix[len(orig_df)-1, 'shaped_reward'] = 100

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000


In [56]:
for i in data2.index:
  if data2.loc[i,"reward_orig"] !=0:
    data2.loc[i,"reward_intm1"] = data2.loc[i,"reward_orig"]
    data2.loc[i,"reward_intm2"] = data2.loc[i,"reward_orig"]


In [65]:
data2.tail(50)

,hosp_id,interval,sofa_total,norepi_equiv,lactate,action,dead_90,reward_orig,reward_intm,state,...,new_next_state,last_record,reward_p15,reward_p5,vaso_change,reward_vaso5,reward_vaso10,reward_vaso15,reward_intm1,reward_intm2
309790,zzl0+TSqL+taprKjH33Rp528C18zk1AVRJpIy67HnCg=,17.0,6.0,0.00,2.0,1.0,0.0,0,0.000000,535,...,229,1,0,0,0.00,0,0,0,0.000000,0.000000
309791,zzl0+TSqL+taprKjH33Rp9UKEu30PFn7fgevBiP09VE=,8.0,4.0,0.00,2.5,16.0,0.0,0,-0.234935,743,...,364,0,0,0,0.00,0,0,0,-0.519751,-0.644751
309792,zzl0+TSqL+taprKjH33Rp9UKEu30PFn7fgevBiP09VE=,9.0,5.0,0.00,2.7,11.0,0.0,0,-1.541462,28,...,193,0,0,0,0.00,0,0,0,-2.303055,-2.678055
309793,zzl0+TSqL+taprKjH33Rp9UKEu30PFn7fgevBiP09VE=,10.0,8.0,0.00,4.7,16.0,0.0,0,-0.275220,168,...,193,0,0,0,0.00,0,0,0,-0.419751,-0.444751
309794,zzl0+TSqL+taprKjH33Rp9UKEu30PFn7fgevBiP09VE=,11.0,8.0,0.00,4.9,21.0,0.0,0,-0.739517,168,...,193,0,0,0,0.00,0,0,0,-1.745818,-1.620818
309795,zzl0+TSqL+taprKjH33Rp9UKEu30PFn7fgevBiP09VE=,12.0,7.0,0.00,6.6,16.0,0.0,0,-0.035768,168,...,193,0,0,0,0.00,0,0,0,-0.050664,-0.300664
309796,zzl0+TSqL+taprKjH33Rp9UKEu30PFn7fgevBiP09VE=,13.0,9.0,0.00,6.5,21.0,0.0,0,0.247359,168,...,193,0,0,0,0.00,0,0,0,1.049234,1.174234
309797,zzl0+TSqL+taprKjH33Rp9UKEu30PFn7fgevBiP09VE=,14.0,8.0,0.00,6.0,16.0,0.0,0,-0.122711,168,...,193,0,0,0,0.00,0,0,0,0.174336,0.149336
309798,zzl0+TSqL+taprKjH33Rp9UKEu30PFn7fgevBiP09VE=,15.0,8.0,0.00,5.9,21.0,0.0,0,-0.587072,168,...,151,0,0,0,0.00,0,0,0,-1.620818,-1.370818
309799,zzl0+TSqL+taprKjH33Rp9UKEu30PFn7fgevBiP09VE=,16.0,6.0,0.00,7.6,21.0,0.0,0,0.177665,609,...,151,0,-15,-5,0.00,0,0,0,0.958736,0.708736


In [67]:
data2[data2.reward_vaso5==-5]

,hosp_id,interval,sofa_total,norepi_equiv,lactate,action,dead_90,reward_orig,reward_intm,state,...,new_next_state,last_record,reward_p15,reward_p5,vaso_change,reward_vaso5,reward_vaso10,reward_vaso15,reward_intm1,reward_intm2
367,+40TQ91K9R0c0RU/llyphvMHKIX5g0+K8mh31Bb1z7M=,9.0,6.0,2.00,2.5,20.0,0.0,0,-0.166667,19,...,519,0,0,0,1.50,-5,-10,-15,-0.025000,-0.050000
370,+40TQ91K9R0c0RU/llyphvMHKIX5g0+K8mh31Bb1z7M=,12.0,6.0,0.50,2.5,19.0,0.0,0,0.189394,271,...,374,0,0,0,1.50,-5,-10,-15,0.625000,1.250000
679,+7Krm6q8zu2sjdl0xAe+nOM3QcR858banND6lTsu9Io=,16.0,10.0,0.20,1.2,4.0,0.0,0,0.530387,546,...,496,0,0,0,1.20,-5,-10,-15,0.557625,0.532625
955,+BmKTNyKHadHVmUMDrND/qWFnpalsM0KFsl/PUEYgIQ=,7.0,16.0,1.20,15.0,25.0,1.0,0,0.251939,233,...,296,0,0,0,1.20,-5,-10,-15,1.926486,1.901486
971,+BuAcGbzMc/mfSILbVuwDmB/JZ4D9sV3zqTFyX/fKw8=,11.0,12.0,2.20,8.9,15.0,1.0,0,0.705625,226,...,486,0,0,0,1.80,-5,-10,-15,2.053055,2.178055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307867,zYoftZ1NHljB/LtLUcQwqGlsUvigs9IsvdSMxXDYF0M=,18.0,11.0,1.20,2.9,25.0,0.0,0,0.000000,476,...,318,1,0,0,1.20,-5,-10,-15,0.000000,0.000000
307919,za9vDIkrZKOcO2N/slG/JJf7UHnX4wKboGA0lQRs5q0=,7.0,11.0,1.20,11.2,10.0,1.0,0,0.056409,309,...,422,0,0,0,1.20,-5,-10,-15,0.833736,0.458736
308556,zguqA+7IERik9KejbrtfEdyxS5mkYBUfagDysFXKyys=,8.0,16.0,2.20,3.7,25.0,1.0,0,-1.015428,25,...,271,0,0,0,1.55,-5,-10,-15,-1.440904,-0.940904
308784,zkuCMAWZ2sN5nKdjRLNSbPiAfM8iseEjnce05zpeKE0=,7.0,14.0,1.60,1.3,25.0,0.0,0,-0.279251,445,...,608,0,0,0,1.20,-5,-10,-15,-0.144751,0.105249


In [70]:
data2 =data2[['hosp_id', 'interval','sofa_total','norepi_equiv','lactate','action', 'dead_90', 'reward_orig',
        'state', 'next_state',"new_state","new_next_state",'last_record', 'vaso_change',"reward_intm1","reward_intm2",
              "reward_vaso5","reward_vaso10","reward_vaso15"]]

In [71]:
data2.head()

,hosp_id,interval,sofa_total,norepi_equiv,lactate,action,dead_90,reward_orig,state,next_state,new_state,new_next_state,last_record,vaso_change,reward_intm1,reward_intm2,reward_vaso5,reward_vaso10,reward_vaso15
0,++Q7obcTFvuaocJ2OPM9rrp0O22cEI0Pes4pfPeZu74=,6.0,5.0,0.0,1.7,1.0,0.0,0,669,669,67,550,0,0.0,-0.074336,0.050664,0,0,0
1,++Q7obcTFvuaocJ2OPM9rrp0O22cEI0Pes4pfPeZu74=,7.0,4.0,0.0,1.8,1.0,0.0,0,669,669,550,67,0,0.0,1.199099,1.324099,0,0,0
2,++Q7obcTFvuaocJ2OPM9rrp0O22cEI0Pes4pfPeZu74=,8.0,3.0,0.0,1.2,1.0,0.0,0,669,669,67,67,0,0.0,-1.457596,-1.482596,0,0,0
3,++Q7obcTFvuaocJ2OPM9rrp0O22cEI0Pes4pfPeZu74=,9.0,3.0,0.0,2.1,1.0,0.0,0,669,669,67,67,0,0.0,-0.025000,-0.050000,0,0,0
4,++Q7obcTFvuaocJ2OPM9rrp0O22cEI0Pes4pfPeZu74=,10.0,3.0,0.0,2.1,1.0,0.0,0,669,669,67,607,0,0.0,-0.025000,-0.050000,0,0,0


In [72]:
data2.to_csv("data/data.csv")

In [9]:
data2= pd.read_csv("data/data.csv")

In [34]:
data2 = pd.read_csv(r"data\data_cont_vaso_reward.csv")

In [22]:
np.quantile(data2[data2.vaso_change!=0].vaso_change,0.99)

2.0

In [38]:
data2[data2.vaso_change!=0].vaso_change.value_counts()

0.0400    4783
0.1000    4380
0.0500    2141
0.2000    2121
0.0200    2041
          ... 
2.0700       1
0.4350       1
1.7600       1
0.4975       1
2.4750       1
Name: vaso_change, Length: 1251, dtype: int64

In [2]:
import numpy as np
0.5-np.tanh(0.45)*5

-1.6094950262500398

In [31]:
0.5-np.tanh(1)*10

-7.1159415595576485

In [35]:
data2['reward_vaso1']=0
for i in data2.index:
  data2.loc[i,'reward_vaso1']=0.5-np.tanh(data2.loc[i,'vaso_change'])*5
for i in data2.index:
  if data2.loc[i,"reward_orig"] !=0:
    data2.loc[i,"reward_vaso1"] = data2.loc[i,"reward_orig"]
   

In [36]:
data2['reward_vaso2']=0
for i in data2.index:
  data2.loc[i,'reward_vaso2']=0.5-np.tanh(data2.loc[i,'vaso_change'])*10
for i in data2.index:
  if data2.loc[i,"reward_orig"] !=0:
    data2.loc[i,"reward_vaso2"] = data2.loc[i,"reward_orig"]
   

In [47]:
data2['reward_vaso3']=0
for i in data2.index:
  data2.loc[i,'reward_vaso3']=0.25-np.tanh(data2.loc[i,'vaso_change'])*2
for i in data2.index:
  if data2.loc[i,"reward_orig"] !=0:
    data2.loc[i,"reward_vaso3"] = data2.loc[i,"reward_orig"]
   

In [48]:
data2.tail(50)

,hosp_id,interval,sofa_total,norepi_equiv,lactate,action,dead_90,reward_orig,state,next_state,...,vaso_change,reward_intm1,reward_intm2,reward_vaso5,reward_vaso10,reward_vaso15,reward_vaso,reward_vaso1,reward_vaso2,reward_vaso3
309790,zzl0+TSqL+taprKjH33Rp528C18zk1AVRJpIy67HnCg=,17.0,6.0,0.00,2.0,1.0,0.0,0,535,743,...,0.00,0.000000,0.000000,0,0,0,0.500000,0.500000,0.500000,0.250000
309791,zzl0+TSqL+taprKjH33Rp9UKEu30PFn7fgevBiP09VE=,8.0,4.0,0.00,2.5,16.0,0.0,0,743,28,...,0.00,-0.519751,-0.644751,0,0,0,0.500000,0.500000,0.500000,0.250000
309792,zzl0+TSqL+taprKjH33Rp9UKEu30PFn7fgevBiP09VE=,9.0,5.0,0.00,2.7,11.0,0.0,0,28,168,...,0.00,-2.303055,-2.678055,0,0,0,0.500000,0.500000,0.500000,0.250000
309793,zzl0+TSqL+taprKjH33Rp9UKEu30PFn7fgevBiP09VE=,10.0,8.0,0.00,4.7,16.0,0.0,0,168,168,...,0.00,-0.419751,-0.444751,0,0,0,0.500000,0.500000,0.500000,0.250000
309794,zzl0+TSqL+taprKjH33Rp9UKEu30PFn7fgevBiP09VE=,11.0,8.0,0.00,4.9,21.0,0.0,0,168,168,...,0.00,-1.745818,-1.620818,0,0,0,0.500000,0.500000,0.500000,0.250000
309795,zzl0+TSqL+taprKjH33Rp9UKEu30PFn7fgevBiP09VE=,12.0,7.0,0.00,6.6,16.0,0.0,0,168,168,...,0.00,-0.050664,-0.300664,0,0,0,0.500000,0.500000,0.500000,0.250000
309796,zzl0+TSqL+taprKjH33Rp9UKEu30PFn7fgevBiP09VE=,13.0,9.0,0.00,6.5,21.0,0.0,0,168,168,...,0.00,1.049234,1.174234,0,0,0,0.500000,0.500000,0.500000,0.250000
309797,zzl0+TSqL+taprKjH33Rp9UKEu30PFn7fgevBiP09VE=,14.0,8.0,0.00,6.0,16.0,0.0,0,168,168,...,0.00,0.174336,0.149336,0,0,0,0.500000,0.500000,0.500000,0.250000
309798,zzl0+TSqL+taprKjH33Rp9UKEu30PFn7fgevBiP09VE=,15.0,8.0,0.00,5.9,21.0,0.0,0,168,609,...,0.00,-1.620818,-1.370818,0,0,0,0.500000,0.500000,0.500000,0.250000
309799,zzl0+TSqL+taprKjH33Rp9UKEu30PFn7fgevBiP09VE=,16.0,6.0,0.00,7.6,21.0,0.0,0,609,609,...,0.00,0.958736,0.708736,0,0,0,0.500000,0.500000,0.500000,0.250000


In [41]:
data2.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'hosp_id', 'interval',
       'sofa_total', 'norepi_equiv', 'lactate', 'action', 'dead_90',
       'reward_orig', 'state', 'next_state', 'new_state', 'new_next_state',
       'last_record', 'vaso_change', 'reward_intm1', 'reward_intm2',
       'reward_vaso5', 'reward_vaso10', 'reward_vaso15', 'reward_vaso',
       'reward_vaso1', 'reward_vaso2'],
      dtype='object')

In [42]:
data2 = data2.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'])

In [55]:
data2.to_csv("data/data_cont_vaso_reward.csv",index=False)

In [1]:
import pandas as pd
data2 = pd.read_csv("data/data_cont_vaso_reward.csv")

In [3]:
import numpy as np
c03 = -0.1/3
c13 = -0.5/3
c23 = -2
data2['reward_intm3'] = 0
for i in data2.index:
    if i == 0:
        continue
    if data2.loc[i, 'hosp_id'] == data2.loc[i-1, 'hosp_id']:
        sofa_cur = data2.loc[i,'sofa_total']
        sofa_prev = data2.loc[i-1,'sofa_total']
        lact_cur = data2.loc[i,'lactate']
        lact_prev = data2.loc[i-1,'lactate']
        reward3 = 0

        if sofa_cur == sofa_prev and sofa_cur != 0:
            reward3 += c03
            
        reward3 += c13*(sofa_cur-sofa_prev)
        reward3 += c23*np.tanh(lact_cur - lact_prev)
        
        data2.loc[i-1,'reward_intm3'] = reward3



        
    if i % 10000 == 0:
        print(i)
for i in data2.index:
  if data2.loc[i,"reward_orig"] !=0:
    data2.loc[i,"reward_intm3"] = data2.loc[i,"reward_orig"]

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000


In [4]:
data2.to_csv("data/data_cont_vaso_reward.csv",index=False)